# RNN 영화 리뷰 긍정, 부정
* kaggle - 영화 리뷰 데이터
> https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
* 긍정인지, 부정인지 학습

In [43]:
!pip install --q ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 8.46 s (started: 2024-07-02 05:10:45 +00:00)


In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

time: 650 µs (started: 2024-07-02 05:10:53 +00:00)


In [45]:
!gdown https://drive.google.com/uc?id=1_LFoolJeek9E1Cug1LSdh6EqxlZrA1eu

Downloading...
From (original): https://drive.google.com/uc?id=1_LFoolJeek9E1Cug1LSdh6EqxlZrA1eu
From (redirected): https://drive.google.com/uc?id=1_LFoolJeek9E1Cug1LSdh6EqxlZrA1eu&confirm=t&uuid=f4d5cedf-70c4-42ed-8311-6fa43e60f8d4
To: /content/movie_review.zip
100% 26.9M/26.9M [00:00<00:00, 64.4MB/s]
time: 1.92 s (started: 2024-07-02 05:10:53 +00:00)


In [46]:
!unzip /content/movie_review.zip

Archive:  /content/movie_review.zip
replace testData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testData.tsv            
replace trainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: trainData.tsv           
time: 26.5 s (started: 2024-07-02 05:10:55 +00:00)


In [47]:
train = pd.read_csv('/content/trainData.tsv', delimiter='\t')
test = pd.read_csv('/content/testData.tsv', delimiter='\t')

train.shape, test.shape

((25000, 3), (25000, 2))

time: 1.41 s (started: 2024-07-02 05:11:22 +00:00)


In [48]:
train.iloc[0].review

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

time: 9.56 ms (started: 2024-07-02 05:11:23 +00:00)


### 전처리

In [49]:
# 어떻게 처리할지 하나씩 테스트
# HTML 태그 삭제
from bs4 import BeautifulSoup

bs = BeautifulSoup(train.iloc[0].review, 'html.parser')
bs.text

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

time: 7.65 ms (started: 2024-07-02 05:11:23 +00:00)


In [50]:
# 숫자, 기호 삭제 - 정규 표현식
import re

cleaned = re.sub(r'[^a-zA-Z]', ' ', bs.text) # a-z, A-Z
cleaned

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

time: 4.68 ms (started: 2024-07-02 05:11:23 +00:00)


In [51]:
# 대문자 -> 소문자
cleaned = cleaned.lower()
cleaned


'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

time: 7.11 ms (started: 2024-07-02 05:11:23 +00:00)


In [52]:
# 블용어(stopwords) -> 문장에서 많이 사용되지만, 특별한 의미가 없는 것을 처리
# 데이터로 간주하기 X
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

time: 11.5 ms (started: 2024-07-02 05:11:23 +00:00)


In [53]:
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

time: 1.43 ms (started: 2024-07-02 05:11:23 +00:00)


In [54]:
for word in cleaned.split():
  if word in eng_stopwords:
    print(word)

with
all
this
down
at
the
with
i
ve
to
his
the
here
and
there
the
and
again
i
just
to
a
into
this
who
i
was
in
the
just
to
up
my
he
is
or
is
which
i
to
at
the
when
it
was
some
of
it
has
about
s
the
and
the
of
are
m
but
of
this
is
all
about
so
you
in
then
you
are
to
this
and
it
some
an
for
to
the
of
this
but
and
most
of
his
that
he
it
for
the
which
if
is
of
him
the
when
it
is
only
on
for
or
so
the
and
is
as
a
all
why
he
so
is
me
because
his
s
that
he
to
it
is
he
who
is
so
i
he
just
s
of
in
this
into
a
and
a
and
the
the
have
had
the
of
a
when
it
to
the
as
with
a
of
them
a
this
is
for
who
on
or
which
i
is
most
if
not
then
it
does
and
off
a
and
s
in
this
is
a
is
of
the
most
to
this
but
is
he
with
all
the
i
ve
this
i
don
t
because
can
be
i
this
for
a
he
is
an
but
or
of
the
most
i
he
is
not
the
time: 73.4 ms (started: 2024-07-02 05:11:23 +00:00)


In [55]:
# 함수로 제작
def preprocess(sentence):
    soup = BeautifulSoup(sentence, 'html.parser')
    cleaned = re.sub(r'[^a-zA-Z]', ' ', soup.text)
    cleaned = cleaned.lower()
    cleaned = [word for word in cleaned.split() if word not in eng_stopwords]
    return ' '.join(cleaned)

time: 1.09 ms (started: 2024-07-02 05:11:23 +00:00)


In [56]:
preprocess(train.iloc[0].review)

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

time: 13.9 ms (started: 2024-07-02 05:11:23 +00:00)


In [57]:
train_clean = train['review'].apply(preprocess) # 각 데이터마다 해당 함수가 적용
train_clean.head()

<ipython-input-55-f7ef895b5bc9>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence, 'html.parser')


0    stuff going moment mj started listening music ...
1    classic war worlds timothy hines entertaining ...
2    film starts manager nicholas bell giving welco...
3    must assumed praised film greatest filmed oper...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

time: 21.2 s (started: 2024-07-02 05:11:23 +00:00)


## Tokenizer

In [58]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_clean)

time: 3.2 s (started: 2024-07-02 05:11:44 +00:00)


In [59]:
# 단어가 몇 개 있는지 확인(단어사전)
len(tokenizer.word_index)

74066

time: 7.38 ms (started: 2024-07-02 05:11:48 +00:00)


In [60]:
tokenizer.word_index['odd']

874

time: 2.87 ms (started: 2024-07-02 05:11:48 +00:00)


## 학습용, 검증 데이터 분리

In [61]:
# 정답 따로 끊어두기
train_label = train['sentiment']
train_label.head()

0    1
1    1
2    0
3    0
4    1
Name: sentiment, dtype: int64

time: 5.6 ms (started: 2024-07-02 05:11:48 +00:00)


In [62]:
from sklearn.model_selection import train_test_split

train_sentence, val_sentence, train_label, val_label = train_test_split(train_clean, train_label, test_size=0.2, random_state=42)
train_sentence.shape, val_sentence.shape

((20000,), (5000,))

time: 11.2 ms (started: 2024-07-02 05:11:48 +00:00)


In [63]:
train_sentence.head()

23311    movie plain dumb casting ralph meeker mike ham...
23623    dahmer young confused man dahmer confusing mov...
1020     may saints preserve us movie going help someon...
12645    combination reading novella viewing film inspi...
1533     daniel day lewis left foot gives us one best p...
Name: review, dtype: object

time: 4.54 ms (started: 2024-07-02 05:11:48 +00:00)


In [64]:
# val sentence를 rnn 데이터 형식으로 만듦
# 시퀀스 - 영어 단어 하나하나를 숫자로 만듦
train_sequence = tokenizer.texts_to_sequences(train_sentence)
val_sequence = tokenizer.texts_to_sequences(val_sentence)

time: 2.02 s (started: 2024-07-02 05:13:49 +00:00)


In [68]:
print(train_sequence[0]) # list로 바뀜

[2, 887, 842, 821, 3003, 14250, 1710, 3772, 24779, 1175, 3, 3297, 1485, 8391, 1710, 3772, 4, 1086, 1052, 168, 26856, 869, 29536, 7405, 160, 1167, 177, 3134, 589, 3772, 1861, 80, 819, 15, 726, 3, 375, 3772, 10390, 3226, 16, 357, 13, 842, 15, 134, 124, 4, 713, 1160, 16758, 3750, 1110, 398, 2, 99, 173, 4304, 178, 155, 72211, 685, 72212, 1673, 16, 118, 2316, 8277, 1200, 53, 2209, 1861, 7592, 34495, 72213, 482, 72214, 1144, 246, 2517, 2191, 2808, 22, 366, 246, 2497, 53, 6810, 205, 138, 839, 87, 17142, 3750, 1110, 16, 118, 6049, 3772, 3911, 4265, 1411, 450, 415, 4265, 172, 1374, 122, 363, 118, 1094, 112, 246, 2265, 1364, 1580, 1797, 16, 101, 32, 170, 106, 30, 614]
time: 2.23 ms (started: 2024-07-02 05:15:06 +00:00)


In [ ]:
# 150개 정도로 정함

## 패딩

In [89]:
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(train_sequence,
              maxlen=150,
              padding='pre',
              truncating='pre')

val_padded = pad_sequences(val_sequence,
              maxlen=150,
              padding='pre',
              truncating='pre')

train_padded.shape, val_padded.shape

((20000, 150), (5000, 150))

time: 394 ms (started: 2024-07-02 05:32:42 +00:00)


## 모델

In [91]:
print(type(train_padded))
train_label = train_label.to_numpy()
val_label = val_label.to_numpy()
train_label, val_label

<class 'numpy.ndarray'>


(array([0, 0, 0, ..., 1, 1, 0]), array([0, 1, 0, ..., 0, 0, 0]))

time: 18.9 ms (started: 2024-07-02 05:32:52 +00:00)


In [92]:
EMBEDDING_DIM = 300
VAOCA_SIZE = len(tokenizer.word_index)
VAOCA_SIZE

74066

time: 4.16 ms (started: 2024-07-02 05:32:56 +00:00)


In [94]:
from keras import layers

model = keras.Sequential([
    layers.Embedding(VAOCA_SIZE, EMBEDDING_DIM, input_length=150), # input_length : 데이터가 한번에 몇개? max가 150개
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(128),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

time: 1.11 s (started: 2024-07-02 05:33:03 +00:00)


In [95]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 150, 300)          22219800  
                                                                 
 lstm_6 (LSTM)               (None, 150, 128)          219648    
                                                                 
 lstm_7 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 22575193 (86.12 MB)
Trainable params: 22

In [96]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

time: 14.9 ms (started: 2024-07-02 05:33:07 +00:00)


In [ ]:
EPOCHS = 1
BATCH_SIZE = 32

history = model.fit(train_padded, train_label,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(val_sequence, val_label))

604/625 [===========================>..] - ETA: 31s - loss: 0.3826 - accuracy: 0.8319

In [ ]:
# 테스트 데이터 준비, 전처리 -> 테스트 데이터에 정답값이 없음
# train을 3등분으로 나눠서 test 진행해보기
# 평가